# Section3：軽量化・高速化技術

# 分散深層学習とは
- 深層学習は多くのデータを使⽤したり、パラメータ調整のために多くの時間を使⽤したりするため、⾼速な計算が求められる。
- 複数の計算資源(ワーカー)を使⽤し、並列的にニューラルネットを構成することで、効率の良い学習を⾏いたい。
- データ並列化、モデル並列化、GPUによる⾼速技術は不可⽋である。

※毎年１０倍のデータ量・モデルの複雑になっていっている。計算量が１０倍ずつになっている。コンピューターは１８ヵ月で２倍の性能になっている。

# 3 高速化

## 3.1 データ並列化
- 親モデルを各ワーカーに⼦モデルとしてコピー（ワーカーがコンピューターにあたる）
- データを分割し、各ワーカーごとに計算させる
- データ並列化は各モデルのパラメータの合わせ⽅で、同期型か⾮同期型か決まる。

### データ並列化: 同期型
パラメータ更新の流れ：各ワーカーが計算が終わるのを待ち、全ワーカーの勾配が出たところで勾配の平均を計算し、親モデルのパラメータを更新する。


### データ並列化: ⾮同期型
パラメータ更新の流れ：各ワーカーはお互いの計算を待たず、各子モデルごとに更新を行う。学習が終わった子モデルはパラメータサーバにPushされる。新たに学習を始める時は、パラメータサーバからPopしたモデルに対して学習していく。

### 同期型と⾮同期型の⽐較
- 処理のスピードは、お互いのワーカーの計算を待たない⾮同期型の⽅が早い。
- ⾮同期型は最新のモデルのパラメータを利⽤できないので、学習が不安定になりやすい。
 - ⇒Stale Gradient Problem
- 現在は同期型の⽅が精度が良いことが多いので、主流となっている。

## 3.2 モデル並列化
- 親モデルを各ワーカーに分割し、それぞれのモデルを学習させる。全てのデータで学習が終わった後で、⼀つのモデルに復元。
- モデルが⼤きい時はモデル並列化を、データが⼤きい時はデータ並列化をすると良い。
- モデルのパラメータ数が多いほど、スピードアップの効率も向上する。

## 3.3 GPUによる⾼速化
-  GPGPU (General-purpose on GPU)
 - 元々の使⽤⽬的であるグラフィック以外の⽤途で使⽤されるGPUの総称
- CPU
 - ⾼性能なコアが少数
 - 複雑で連続的な処理が得意
- GPU
 - ⽐較的低性能なコアが多数
 - 簡単な並列処理が得意
 - ニューラルネットの学習は単純な⾏列演算が多いので、⾼速化が可能

## 3.4 モデルの軽量化
モデルの精度を維持しつつパラメータや演算回数を低減する⼿法の総称
- ⾼メモリ負荷 ⾼い演算性能が求められる 通常は
- 低メモリ 低演算性能での利⽤が必要とされるIotなど

代表的な⼿法 として下記の 3 つがある
- 量⼦化
- 蒸留
- プルーニング

### 3.4.1 量⼦化
- ネットワークが⼤きくなると⼤量のパラメータが必要なり学習や推論に多くのメモリと演算処理が必要
 - 通常のパラメータの 64 bit 浮動⼩数点を 32 bit など下位の精度に落とすことでメモリと演算処理の削減を⾏う

#### 利点
- 計算の⾼速化
 - 倍精度演算(64 bit)と単精度演算(32 bit)は演算性能が⼤きく違うため、量⼦化により精度を落とすことによりより多くの計算をすることができる。
 - 深層学習で⽤いられる NVIDIA 社製の GPU の性能は下記のようになる。
- 省メモリ化
 - ニューロンの重みを浮動小数点のbitを少なくし有効桁数を下げることで、ニューロンのメモリサイズを小さくすることができ、メモリ使用量を抑えることができる。

#### ⽋点
- 精度の低下
 - ニューロンが表現できる少数の有効桁数が小さくなるため

### 3.4.2 蒸留

#### モデルの簡約化
学習済みの精度の⾼いモデルの知識を軽量なモデルへ継承させる。知識の継承により、軽量でありながら複雑なモデルに匹敵する精度のモデルを得ることが期待できる。

#### 教師モデルと⽣徒モデル
- 蒸留は教師モデルと⽣徒モデルの2つで構成される
- 教師モデルの重みを固定し⽣徒モデルの重みを更新していく誤差は教師モデルと⽣徒モデルのそれぞれの誤差を使い重みを更新していく

##### 教師モデル
予測精度の⾼い、複雑なモデルやアンサンブルされたモデル

##### ⽣徒モデル
教師モデルをもとに作られる軽量なモデル

#### 蒸留の利点
蒸留によって少ない学習回数でより精度の良いモデルを作成することができている

### 3.4.3 プルーニング

### 計算の⾼速化
寄与の少ないニューロンの削減を行い、モデルの圧縮を行うことで高速に計算できる。

ニューロンの削減の⼿法は重みが閾値以下の場合ニューロンを削減し、再学習を⾏う。 \
例として、重みが 0.1 以下のニューロンを削減する。